# Image Filters and Basic Feature Detection
*Author: Vladislav Kim*
* [Introduction](#intro)
* [Edge enhancement](#edge)
* [Spot detection](#spot)
* [Fourier domain](#fourier)

## Introduction
<a id="intro"></a> 
In addition to denoising, thresholding and background subtraction that were covered in the [previous](https://github.com/vladchimescu/bioimg/blob/master/Jupyter/2-image-transformation.ipynb) notebook, there are a number of feature enhancing image transformations that can be useful for microscopy image analysis. In many applications we are interested in finding edges or enhancing object boundaries. Also common in microscopy are spherical shapes, e.g. nuclei, which can be detected using Laplace-of-Gaussian (LoG) operator.

In [ ]:
# load third-party Python modules
import javabridge
import bioformats as bf
import skimage
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

import sys
sys.path.append('..')

javabridge.start_vm(class_path=bf.JARS)

Load an image stack of leukemia cells:

In [ ]:
from base.utils import load_imgstack
imgstack = load_imgstack(fname="data/BiTE/Tag2-r04c02f1.tiff")

In [ ]:
mip=np.max(imgstack, axis=0)

In [ ]:
from base.plot import plot_channels
channels = ['PE', 'Calcein', 'Hoechst', 'APC']
plot_channels([mip[:,:,i]**0.5 for i in range(4)],
              nrow=1, ncol=4, titles=channels)

## Edge enhancement
<a id="edge"></a> 
We will apply edge enhancement in APC channel with stained surface markers:

In [ ]:
apc = mip[:,:,3]**0.3

Edges are boundaries or interfaces of objects, which can be detected using gradient operators.

In [ ]:
from skimage.filters import sobel

In [ ]:
plot_channels([apc, sobel(apc)],
             nrow=1, ncol=2,
             titles=['Original', 'Sobel edge enhancement'],
             cmap='gray', scale_x=7, scale_y=7)

In [ ]:
plt.figure(figsize=(6,6))
sn.distplot(apc, kde=False, label='Original image')
sn.distplot(sobel(apc), kde=False, label='Sobel edge enhancement')
plt.xlabel('Intensity')
plt.legend()

In [ ]:
# compute x and y components of the gradient
g_x, g_y = np.gradient(apc)
# magnitude of the gradient
g_norm = np.sqrt(g_x**2 + g_y**2)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(g_norm, cmap='gray')
plt.title('Gradient image')
plt.axis('off')

## Spot detection
<a id="spot"></a> 

## Fourier domain
<a id="fourier"></a> 